# Get new titles by keyword, MWS
Search MWS for keywords in list. 
Save all ASINs appearing in results, as well as their price, salesrank, and publication date. 
Then sort lists into interesting and not interesting. 


In [1]:
import pandas as pd
import random
import mws
import datetime
import time
from tenacity import *  

import booksnn as bnn

#kinda unnecessary so far
#from mws_functions import * 

date = datetime.datetime.now().strftime("%Y_%m_%d")

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


#### Import list of keywords

In [2]:
# Load list of keywords
keywords = list(pd.read_table('keywords_more_likely_textbooks.txt').columns)

# display random 10 entries in list
i = random.randint(0, len(keywords)-10)
keywords[i+0:i+10]

/home/lynx/anaconda3/envs/booksnn/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['Afghan & Iraq Wars',
 'Aesthetics',
 'Aerospace',
 'Aeronautics & Astronautics',
 'Aerodynamics',
 'Aerial',
 'Advertising',
 'Adventure',
 'Advanced Placement',
 'Adult Ministry']

#### Connect to MWS and create productsAPI object

#### Load current ASINs in database 

In [4]:
asins_in_db = bnn.get_asin_list()

#### Loop through Keywords
For each found product, save ASIN to one of two lists if it's not already tracked in database. 

In [19]:
def keyword_lookup(query):
    ''' Perform keyword lookup and return list of 10 matching ASINS, as well as:
        Title, publication date, listprice, salesrank
        and
        requests remaining, time to full reset
    '''
    result_list = [] #will be list of tuples of product_info
    
    response = products_api.list_matching_products(marketplace_id, query, context_id)
    
    # Get info about requests reamining
    #requests_max = response.headers['x-mws-quota-max']
    requests_remaining = response.headers['x-mws-quota-remaining']
    reset_time = response.headers['x-mws-quota-resetsOn']
    amztime = response.headers['X-Amz-Date'] 
    reset_time_obj = datetime.datetime.strptime(reset_time[0:19],'%Y-%m-%dT%H:%M:%S')
    amz_time_obj = datetime.datetime.strptime(amztime,'%a, %d %b %Y %H:%M:%S %Z')
    td = reset_time_obj - amz_time_obj
    seconds_to_full_reset = td.total_seconds()
    request_info = (requests_remaining,seconds_to_full_reset)
    
    # Get product info
    for product in response.parsed.Products.Product: 
        asin = product.Identifiers.MarketplaceASIN.ASIN
        title = product.AttributeSets.ItemAttributes.Title
        
        try: 
            pubdate = product.AttributeSets.ItemAttributes.PublicationDate
            pubdate = datetime.datetime(int(pubdate[0:4]),int(pubdate[5:7]),int(pubdate[8:10]))
        except: 
            pubdate = datetime.datetime.now().strftime("%Y-%m-%d") #placeholder

        try:
            listprice = float(product.AttributeSets.ItemAttributes.ListPrice.Amount)
        except: 
            listprice = 0 #placeholder
            
        try:
            salesrank = int(product.SalesRankings.SalesRank[0].Rank)
        except: 
            salesrank = 30000000 #assume book very unpopular if no listed rank
            
        try: 
            num_pages = int(product.AttributeSets.ItemAttributes.NumberOfPages)
        except:
            num_pages = 0
            
        product_info = {'asin': asin,
                        'title': title,
                        'pubdate': pubdate,
                        'listprice': listprice,
                        'salesrank': salesrank,
                        'num_pages': num_pages}
        result_list.append(product_info)
    
    return result_list, request_info
    

In [23]:
'''def get_ASINs_from_keywords():
    return
'''
#print('Expect the queries to take at least '+ str(len(keywords)/720) + ' hours')

#new_asins_df = pd.DataFrame(columns=[])
#skipped_keywords = []
i=0
for query in keywords:
#for query in skipped_keywords:
    time.sleep(5+(random.randint(0,10)/10)) #sleep between 5 and 6sec to limit rate (refreshes 1 token/5sec)
    try: 
        (result_list, request_info) = keyword_lookup(query)
        print(query + ', requests remaining: ' + str(request_info[0]))
        new_asins_df = new_asins_df.append(result_list,ignore_index=True)
    except: #hit the request limit - wait until next available time and restart
        print('timeout of ' + str(request_info[1]) + ' seconds')
        time.sleep(request_info[1]) # put in sleep time here
        skipped_keywords.append(query)
    i+=1
    if i==50:
        # save data to csv in case of interruption
        new_asins_df.to_csv('./asin_lists/asins_from_keywords_'+date+'.csv',index=False,sep='\t')
        i=0

print('skipped keywords:')
print(skipped_keywords)
# Go back and run through skipped keywords as well

Winter Sports, requests remaining: 612.0
Windows XP, requests remaining: 612.0
Volcanology, requests remaining: 612.0
Unemployment, requests remaining: 612.0
Thesauruses, requests remaining: 612.0
Space Marine, requests remaining: 611.0
Retailing, requests remaining: 611.0
Remedies, requests remaining: 611.0
Reading Skills, requests remaining: 610.0
Punk, requests remaining: 609.0
Psychology & Christianity, requests remaining: 608.0
Microprocessor Design, requests remaining: 606.0
Lacrosse, requests remaining: 604.0
Illustration, requests remaining: 603.0
Fiber Optics, requests remaining: 603.0
skipped keywords:
['Winter Sports', 'Windows XP', 'Volcanology', 'Unemployment', 'Thesauruses', 'Space Marine', 'Retailing', 'Remedies', 'Reading Skills', 'Punk', 'Psychology & Christianity', 'Microprocessor Design', 'Lacrosse', 'Illustration', 'Fiber Optics']


In [25]:
# Final save
new_asins_df.to_csv('./asin_lists/asins_from_keywords_'+date+'.csv',index=False,sep='\t')

/home/lynx/anaconda3/envs/booksnn/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



#### Check criteria for interesting products
Save separate list of only interesting ASINs with some details. 

Criteria:
* At least 6mo since pub date. (Why? New items have messed up salesrank, see below. Also need enough history for valid analysis.) (Consider also removing very old books)
* List price > $25
* Sales rank < 2M
* num_pages > 0
* Not already in local database

Note - shitty listings are missing many parameters, e.g. [this one](https://www.amazon.com/ZOROASTRIANISM-History-Hoshang-J-Khambatta/dp/0926592270/). 

Observation - books that are very new (days, weeks) have an unusually low sales rank because salesrank is adjusted to age. E.g. 1 sale in first month of book's publication is pretty pathetic, but it seems to have comparable salesrank to a book that sells one copy weekly, consistently over time. 




In [29]:
#check listprice and salesrank
interesting_df = new_asins_df[(new_asins_df['listprice'] > 25) & (new_asins_df['salesrank'] < 2000000)]
#check older than 6mo
interesting_df = interesting_df[interesting_df['pubdate'] < datetime.datetime.now()-datetime.timedelta(days = 180)]

#check not in existing list
#interesting_df = interesting_df[~interesting_df['asin'].isin(asins_in_db)]

print('Number of ASINs found: ' + str(len(new_asins_df)))
print('Number of them that are interesting: ' + str(len(interesting_df)))

Number of ASINs found: 27634
Number of them that are interesting: 9785


#### Save interesting list to tab-delimited file
Remember to use tab separator because commas in titles mess things up. 

In [30]:
interesting_df.to_csv('./asin_lists/asins_from_keywords_interesting_'+date+'.csv',index=False,sep='\t')

/home/lynx/anaconda3/envs/booksnn/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
